In [10]:
import trimesh
import glob
import numpy as np
path = "/home/yigittunali/Documents/Projects/ml3d/objfiles/"

objfiles = glob.glob(path + "*.obj")


In [6]:
def parse_scene(scene):
    try:

        keys = list(scene.geometry.keys())
        g_verts = []
        g_colors =[]
        for k in keys:
            print(k)
            colors = scene.geometry[k].visual.to_color().vertex_colors
            verts = np.asarray(scene.geometry[k].vertices)
            g_colors.append(colors)
            g_verts.append(verts)
        print("This is a scene, there are multiple meshes inside, and they are concatenated")
    except:
        g_verts = []
        g_colors =[]
        colors = scene.visual.to_color().vertex_colors
        verts = np.asarray(scene.vertices)
        g_colors.append(colors)
        g_verts.append(verts)
        print("This is a single mesh")
    return np.concatenate(g_verts),np.concatenate(g_colors)[:,:3]

In [14]:
meshes = []
for o in objfiles:
    f = open(o,"r") 
    scene = trimesh.load_mesh(f,"obj")
    verts,colors = parse_scene(scene)
    np.save(o[:o.find(".obj")] + ".npy",np.stack([verts,colors],axis=1))
    #print("Printing verts and colors as sanity check ",verts,colors)

, and they are concatenated
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
None_9HOhOLWq8LLl
None_9XPQCZbd3QPA
This is a scene, there are multiple meshes inside, and they are concatenated
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
None_BXS4kqn5fxrq
None_BXSoYBD0sq66
This is a scene, there are multiple meshes inside, and they are concatenated
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mesh
This is a single mes

In [15]:
npys = glob.glob(path + "*.npy")
len(npys)

1031

In [ ]:
# abla.apply_transform(trimesh.transformations.random_rotation_matrix(np.random.rand(3))).show()